In [1]:
# Author: Syed (codefather)
# Date: 2024-05-28

import matplotlib.pyplot as plt
import tifffile as tiff
import numpy as np
import os
import time
from os.path import join
from pycromanager import Core

# creating required directories
os.makedirs('Autofocus', exist_ok=True)
os.makedirs('Stitches', exist_ok=True)

# core should be global - to be fixed later
core = Core()

c:\Users\barya\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyjavaz\bridge.py:308: UserWarning: Version mistmatch between Java ZMQ server and Python client. 
Java ZMQ server version: 4.0.0
Python client expected version: 5.1.0
 To fix, update to BOTH latest pycromanager and latest micro-manager nightly build
  warnings.warn(


In [2]:
#%% Autofocus_WS_plane

def autofocus_plane(core:Core=None, path="", n_images=60) -> None:
    # core.load_system_configuration("core.config_demo.cfg")

    # Record images along z axis
    # core.set_property("TransmittedLamp", "Label", "On")
    # core.set_camera_device("AmScope")
    camera_name = core.get_camera_device()
    # core.set_property(camera_name, "Binning", "1x1")
    # core.set_property(camera_name, "PixelType", "GREY8")
    # core.set_property(camera_name, "ExposureAuto", "0")
    # core.set_serial_port_command("COM13", "CURRENT=0", "\r\n")
    width = core.get_image_width()
    height = core.get_image_height()
    byte_depth = core.get_bytes_per_pixel()
    xystage = core.get_xy_stage_device()
    zstage = core.get_focus_device()

    pos = {'z': core.get_position(zstage)}
    nz = n_images  # number of images
    step = 5
    pos['z'] -= nz * 0.5 * step

    for a in range(nz + 2):
        core.snap_image()
        # core.snap_image()
        # time.sleep(0.05)
        img = core.get_image()
        
        if byte_depth == 1:
            ip = np.reshape(img, (height, width)).astype(np.uint8)
        elif byte_depth == 2:
            ip = np.reshape(img, (height, width)).astype(np.uint16)
        else:
            raise ValueError(f'byte depth should be 1 or 2. byte depth: {byte_depth}')

        if a == 0 or a == 1:
            pre_path = join(path, f"Autofocus/image_{a}.tif")
        else:
            pre_path = join(path, f"Autofocus/image_{a-2}.tif")

        tiff.imwrite(pre_path, ip, photometric='minisblack')
        pos['z'] += step
        # core.set_position(zstage, pos['z'])
        # time.sleep(0.05)

    pos['z'] -= nz * step
    # core.set_position(zstage, pos['z'])

    # Find minimum variance corresponds to in-focus plane
    # core.set_property("TransmittedLamp", "Label", "Off")
    # core.set_camera_device("Andor")
    core.snap_image()

    image = tiff.imread(join(path, "Autofocus/image_0.tif"))
    mean, std = np.mean(image), np.std(image)
    norm_var = std * std / mean

    min_var, min_var_index = norm_var, 0
    for i in range(1, nz):
        image = tiff.imread(join(path, f"Autofocus/image_{i}.tif"))
        mean, std = np.mean(image), np.std(image)
        norm_var = std * std / mean
        if norm_var < min_var:
            min_var, min_var_index = norm_var, i

    pos['z'] += step * (min_var_index) - 10
    # core.set_position(zstage, pos['z'])
    # time.sleep(0.5)


autofocus_plane(core)

In [3]:
#%% image_grid -> cell_filter_isolation

def capture_image_grid(core:Core=None, path:str="", rows:int=4, cols:int=5) -> None:
    width = core.get_image_width()
    height = core.get_image_height()
    byte_depth = core.get_bytes_per_pixel()
    camera_name = core.get_camera_device()

    # core.set_property(camera_name, "Binning", "1x1")
    # core.set_property(camera_name, "PixelType", "GREY8")
    # core.set_property(camera_name, "ExposureAuto", "0")
    # core.set_exposure(25)

    xystage = core.get_xy_stage_device()
    pos_x = core.get_x_position(xystage)
    pos_y = core.get_y_position(xystage)

    x_origin = pos_x
    y_origin = pos_y

    # Grid settings
    n, m = cols, rows  
    inc_x = 320  # step size in X direction
    inc_y = 240  # step size in Y direction

    pos_x = pos_x - inc_x * (0.5 * n - 0.5)
    pos_y = pos_y - inc_y * (0.5 * m - 0.5)
    startx, starty = pos_x, pos_y
    cc = 0

    # core.set_xy_position(pos_x, pos_y)
    time.sleep(1)

    for a in range(m):
        for b in range(n):
            pos_x = startx + inc_x * b
            pos_y = starty + inc_y * a

            # core.set_xy_position(pos_x, pos_y)
            # time.sleep(0.5)
            core.snap_image()
            # core.snap_image()
            # time.sleep(0.5)

            img = core.get_image()

            if byte_depth == 1:
                ip = np.reshape(img, (height, width)).astype(np.uint8)
            elif byte_depth == 2:
                ip = np.reshape(img, (height, width)).astype(np.uint16)
            else:
                raise ValueError(f'byte depth should be 1 or 2. byte depth: {byte_depth}')

            pre_path = os.path.join(path, f"Stitches/imageXY-{cc}.tif")
            tiff.imwrite(pre_path, ip, photometric='minisblack')
            cc += 1

    # core.set_xy_position(x_origin, y_origin)


capture_image_grid(core)

In [4]:
import cv2
from skimage.feature import peak_local_max
from skimage import io, img_as_ubyte


def stitch_images(core:Core=None, path:str="", rows:int=4, cols:int=5) -> None:
    n, m = cols, rows 
    input_dir = os.path.join(path, "Stitches")
    image_files = [os.path.join(input_dir, f"imageXY-{i}.tif") for i in range(0, n * m)]
    stitched_image_rows = []

    # stitching all images by row
    for row in range(m):
        stitched_row = None
        for col in range(n):
            image_file = image_files[row*n + col]
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            if stitched_row is None:
                stitched_row = image
            else:
                stitched_row = np.hstack((stitched_row, image))
        stitched_image_rows.append(stitched_row)

    stitched_image = np.vstack(stitched_image_rows)
    stitched_image_path = os.path.join(input_dir, "stitchedImage.tif")
    cv2.imwrite(stitched_image_path, stitched_image)

    stitched_image = img_as_ubyte(stitched_image) # converting to 8-bit unsigned

    # finding maxima
    coordinates = peak_local_max(stitched_image, min_distance=20, threshold_abs=35)

    # saving cell positions
    result_file = os.path.join(input_dir, "cell_positions.txt")
    with open(result_file, 'w') as f:
        for coord in coordinates:
            f.write(f"{coord[1]},{coord[0]}\n")
    
stitch_images(core)